**Troubleshooting Scripts - System Configuration**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**SQL Server Information**

In [ ]:
SELECT 
    @@SERVERNAME AS [Server Name]
    ,@@VERSION AS [SQL Server Version]
    ,SERVERPROPERTY('Edition') AS [Edition]
    ,SERVERPROPERTY('ProductLevel') AS [ProductLevel]				
    ,SERVERPROPERTY('ProductUpdateLevel') AS [ProductUpdateLevel]	
    ,SERVERPROPERTY('ProductVersion') AS [ProductVersion]
    ,SERVERPROPERTY('ProductMajorVersion') AS [ProductMajorVersion]
    ,SERVERPROPERTY('ProductMinorVersion') AS [ProductMinorVersion] 
    ,SERVERPROPERTY('InstanceName') AS [InstanceName]
    ,SERVERPROPERTY('Collation') AS [Collation]    
    ,SERVERPROPERTY('IsIntegratedSecurityOnly') AS [IsIntegratedSecurityOnly]
    ,SERVERPROPERTY('MachineName') AS [MachineName]
    ,SERVERPROPERTY('IsClustered') AS [IsClustered] 
    ,SERVERPROPERTY('ComputerNamePhysicalNetBIOS') AS [ComputerNamePhysicalNetBIOS]
    ,SERVERPROPERTY('ProductBuild') AS [ProductBuild]
    ,SERVERPROPERTY('ProductBuildType') AS [ProductBuildType]			  
    ,SERVERPROPERTY('ProductUpdateReference') AS [ProductUpdateReference] 
    ,SERVERPROPERTY('IsFullTextInstalled') AS [IsFullTextInstalled]
    ,SERVERPROPERTY('FilestreamConfiguredLevel') AS [FilestreamConfiguredLevel]
    ,SERVERPROPERTY('IsHadrEnabled') AS [IsHadrEnabled]
    ,SERVERPROPERTY('HadrManagerStatus') AS [HadrManagerStatus]
    ,SERVERPROPERTY('InstanceDefaultDataPath') AS [InstanceDefaultDataPath]
    ,SERVERPROPERTY('InstanceDefaultLogPath') AS [InstanceDefaultLogPath]
    ,SERVERPROPERTY('InstanceDefaultBackupPath') AS [InstanceDefaultBackupPath]
    ,SERVERPROPERTY('ErrorLogFileName') AS [ErrorLogFileName]
    ,SERVERPROPERTY('BuildClrVersion') AS [Build CLR Version]
    ,SERVERPROPERTY('IsXTPSupported') AS [IsXTPSupported]
    ,SERVERPROPERTY('IsPolybaseInstalled') AS [IsPolybaseInstalled]
    ,SERVERPROPERTY('IsAdvancedAnalyticsInstalled') AS [IsRServicesInstalled]
    ,SERVERPROPERTY('IsTempdbMetadataMemoryOptimized') AS [IsTempdbMetadataMemoryOptimized]    

**Trace Flags**

In [ ]:
DBCC TRACESTATUS(-1);

**OS Information** (SQL Server 2017+)

In [ ]:
SELECT *
FROM sys.dm_os_host_info WITH (NOLOCK) 
OPTION (MAXDOP 1, RECOMPILE); 

**SQL Server H/W Configuration**

In [ ]:
SELECT 
    DATEDIFF(hour, sqlserver_start_time, GETDATE()) AS [Up Time (hrs)]
    , *
FROM sys.dm_os_sys_info WITH (NOLOCK) 
OPTION (MAXDOP 1, RECOMPILE);

**SQL Server Services**

Check the status of Instant File Initialization

In [ ]:
SELECT 
    servicename
    ,instant_file_initialization_enabled
    ,process_id
    ,last_startup_time
    ,startup_type_desc
    ,status_desc
    ,service_account
    ,is_clustered
    ,cluster_nodename
    ,[filename]
FROM 
    sys.dm_server_services WITH (NOLOCK) 
OPTION (MAXDOP 1, RECOMPILE);

**Checking Instant File nitialization** (Old SQL Server Versions)

In [ ]:
-- Older versions
DBCC TRACEON(3004,3605,-1);
GO
CREATE DATABASE Dummy;
GO
EXEC sp_readerrorlog 0,1,N'Dummy';
GO
DROP DATABASE Dummy;
GO
DBCC TRACEOFF(3004,3605,-1);
GO


**NUMA / Schedulers distribution**

Make sure that online schedules have been evenly distributed across NUMA nodes

In [ ]:
SELECT 
    parent_node_id AS [NUMA Node]
    ,COUNT(*) AS [Schedulers]
    ,SUM(IIF(status = N'VISIBLE ONLINE',1,0))
        AS [Online Schedulers]
    ,SUM(IIF(status = N'VISIBLE OFFLINE',1,0))
        AS [Offline Schedulers]
    ,SUM(current_tasks_count) 
        AS [Current Tasks] 
    ,SUM(runnable_tasks_count) 
        AS [Runnable Tasks] 
FROM 
    sys.dm_os_schedulers WITH (NOLOCK)
WHERE 
    status IN (N'VISIBLE ONLINE',N'VISIBLE OFFLINE') 
GROUP BY 
    parent_node_id
OPTION (RECOMPILE, MAXDOP 1);

**Configuration settings**

List of settings is based on SQL Server 2019

In [ ]:
DECLARE
    @defaults TABLE
    (
        name SYSNAME NOT NULL PRIMARY KEY, 
        def_value SQL_VARIANT NOT NULL
    )

INSERT INTO @defaults(name,def_value) 
VALUES
	('access check cache bucket count',0),
	('access check cache quota',0),
	('Ad Hoc Distributed Queries',0),
	('ADR cleaner retry timeout (min)',0),
	('ADR Preallocation Factor',0),
	('affinity I/O mask',0),
	('affinity mask',0),
	('affinity64 I/O mask',0),
	('affinity64 mask',0),
	('Agent XPs',0),
	('allow filesystem enumeration',1),
	('allow polybase export',0),
	('allow updates',0),
	('automatic soft-NUMA disabled',0),
	('backup checksum default',0),
	('backup compression default',0),
	('blocked process threshold (s)',0),
	('c2 audit mode',0),
	('clr enabled',0),
	('clr strict security',1),
	('column encryption enclave type',0),
	('common criteria compliance enabled',0),
	('contained database authentication',0),
	('cost threshold for parallelism',5),
	('cross db ownership chaining',0),
	('cursor threshold',-1),
	('Database Mail XPs',0),
	('default full-text language',1033),
	('default language',0),
	('default trace enabled',1),
	('disallow results from triggers',0),
	('EKM provider enabled',0),
	('external scripts enabled',0),
	('filestream access level',0),
	('fill factor (%)',0),
	('ft crawl bandwidth (max)',100),
	('ft crawl bandwidth (min)',0),
	('ft notify bandwidth (max)',100),
	('ft notify bandwidth (min)',0),
	('hadoop connectivity',0),
	('index create memory (KB)',0),
	('in-doubt xact resolution',0),
	('lightweight pooling',0),
	('locks',0),
	('max degree of parallelism',0),
	('max full-text crawl range',4),
	('max server memory (MB)',2147483647),
	('max text repl size (B)',65536),
	('max worker threads',0),
	('media retention',0),
	('min memory per query (KB)',1024),
	('min server memory (MB)',0),
	('nested triggers',1),
	('network packet size (B)',4096),
	('Ole Automation Procedures',0),
	('open objects',0),
	('optimize for ad hoc workloads',0),
	('PH timeout (s)',60),
	('polybase enabled',0),
	('polybase network encryption',1),
	('precompute rank',0),
	('priority boost',0),
	('query governor cost limit',0),
	('query wait (s)',-1),
	('recovery interval (min)',0),
	('remote access',1),
	('remote admin connections',0),
	('remote data archive',0),
	('remote login timeout (s)',10),
	('remote proc trans',0),
	('remote query timeout (s)',600),
	('Replication XPs',0),
	('scan for startup procs',0),
	('server trigger recursion',1),
	('set working set size',0),
	('show advanced options',0),
	('SMO and DMO XPs',1),
	('tempdb metadata memory-optimized',0),
	('transform noise words',0),
	('two digit year cutoff',2049),
	('user connections',0),
	('user options',0),
	('version high part of SQL Server',0),
	('version low part of SQL Server',0),
	('xp_cmdshell',0);

SELECT
    c.name, c.description, c.value_in_use, c.value
    ,d.def_value, c.is_dynamic, c.is_advanced
FROM
    sys.configurations c JOIN @defaults d ON
        c.name = d.name
WHERE
    c.value_in_use <> d.def_value OR
    c.value <> d.def_value
ORDER BY
    c.name;


**Resource Governor**

Check if it is configured. Take its existence into account during troubleshooting

In [ ]:
SELECT 
    pool_id
    ,name
    ,statistics_start_time
    ,min_memory_percent
    ,max_memory_percent
    ,CONVERT(DECIMAL(26,3),max_memory_kb/1024) AS max_memory_mb
    ,CONVERT(DECIMAL(26,3),used_memory_kb / 1024.) AS used_memory_mb
    ,CONVERT(DECIMAL(26,3),target_memory_kb / 1024.) AS target_memory_mb
	,min_iops_per_volume -- SQL Server 2014+
    ,max_iops_per_volume -- SQL Server 2014+
FROM sys.dm_resource_governor_resource_pools WITH (NOLOCK)
OPTION (RECOMPILE);

**Memory Dumps**

In [ ]:
SELECT *
FROM sys.dm_server_memory_dumps WITH (NOLOCK) 
ORDER BY creation_time DESC 
OPTION (MAXDOP 1, RECOMPILE);